In [3]:
import pandas as pd
from transformers import pipeline

In [2]:
df = pd.read_csv('/home/laajila/mima_newcode/clean_code/outputs/2025-02-05/14-14-20/output_20250205_144847.csv')

In [4]:
df.head(2)

,Dialogue_Generated,symptom,description,meta,language_style,Tone,Detail_level,Enumeration,Explicit_symptom,Spelling_errors
0,"""I'm so scared, I've got these cracks at the ...",Cracking at the corners of the mouth (cheilosi...,Severity,Very severe,Informal Register,Fearful,1,False,True,True
1,"""I got these cracks at the corners of me mout...",Cracking at the corners of the mouth (cheilosi...,Severity,Not sexually active,Vulgar Register,Fearful,2,False,True,True


In [5]:
df.shape

(823, 10)

In [6]:
ner_pipeline = pipeline("ner", model="d4data/biomedical-ner-all", aggregation_strategy="simple")


extracted_data = []
for i , dialogue in enumerate(df['Dialogue_Generated']) :
    ner_results = ner_pipeline(dialogue)
    symptoms = [entity["word"] for entity in ner_results if "symptom" in entity["entity_group"].lower()]
    
    extracted_data.append({
        "Dialogue": dialogue,
        "Real symptoms": df['symptom'][i],
        "Extracted_Symptoms": symptoms
    })
    
df_extracted = pd.DataFrame(extracted_data)
df_extracted.head(5)

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,Dialogue,Real symptoms,Extracted_Symptoms
0,"""I'm so scared, I've got these cracks at the ...",Cracking at the corners of the mouth (cheilosi...,[cracks]
1,"""I got these cracks at the corners of me mout...",Cracking at the corners of the mouth (cheilosi...,[]
2,"""I am fed up with this bloody cracking at the...",Cracking at the corners of the mouth (cheilosi...,[]
3,"""I have been experiencing a cracking at the c...",Cracking at the corners of the mouth (cheilosi...,[cracking]
4,"""I am fed up with this cracking at the corner...",Cracking at the corners of the mouth (cheilosi...,"[fed, cracking]"


In [7]:
nothing_extracted = [1 for i in range(len(df_extracted['Extracted_Symptoms'])) if df_extracted['Extracted_Symptoms'][i]==[]]
len(nothing_extracted)

217

In [8]:
df_extracted

,Dialogue,Real symptoms,Extracted_Symptoms
0,"""I'm so scared, I've got these cracks at the ...",Cracking at the corners of the mouth (cheilosi...,[cracks]
1,"""I got these cracks at the corners of me mout...",Cracking at the corners of the mouth (cheilosi...,[]
2,"""I am fed up with this bloody cracking at the...",Cracking at the corners of the mouth (cheilosi...,[]
3,"""I have been experiencing a cracking at the c...",Cracking at the corners of the mouth (cheilosi...,[cracking]
4,"""I am fed up with this cracking at the corner...",Cracking at the corners of the mouth (cheilosi...,"[fed, cracking]"
...,...,...,...
818,"""I have been experiencing pain and swelling a...",Pain and swelling at injection site,"[pain, swelling]"
819,"""I'm still in a lot of pain and swelling at t...",Pain and swelling at injection site,"[pain, swelling]"
820,"""Oh, the agony! My arm is on fire and twice a...",Pain and swelling at injection site,[]
821,"""I have a lot of pain and swelling at the inj...",Pain and swelling at injection site,"[pain, swelling]"


In [11]:
df_extracted.iloc[820]['Dialogue']

' "Oh, the agony! My arm is on fire and twice as big as it should be. What kind of sadistic monster would do this to me?"'

In [ ]:
classifier = pipeline("text-classification", model="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

extracted_data = []
for _, row in df.iterrows():
    classification_result = classifier(row["Dialogue_Generated"])[0]  # Get first result
    
    extracted_data.append({
        "Dialogue": row["Dialogue_Generated"],
        "Real symptoms": row["symptom"],
        "Predicted_Label": classification_result["label"],  # Example: "has_symptom"
        "Confidence": classification_result["score"]  # Model confidence score
    })

df_extracted = pd.DataFrame(extracted_data)
df_extracted.head()

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

Device set to use cuda:0


,Dialogue,Real symptoms,Predicted_Label,Confidence
0,"""I'm so scared, I've got these cracks at the ...",Cracking at the corners of the mouth (cheilosi...,LABEL_1,0.541197
1,"""I got these cracks at the corners of me mout...",Cracking at the corners of the mouth (cheilosi...,LABEL_1,0.522005
2,"""I am fed up with this bloody cracking at the...",Cracking at the corners of the mouth (cheilosi...,LABEL_1,0.514203
3,"""I have been experiencing a cracking at the c...",Cracking at the corners of the mouth (cheilosi...,LABEL_0,0.506268
4,"""I am fed up with this cracking at the corner...",Cracking at the corners of the mouth (cheilosi...,LABEL_1,0.501638


In [30]:
classification_result

{'label': 'LABEL_1', 'score': 0.5154854655265808}